# Principal Component Analysis

Using a prinicpal component analysis to reduce the number of variables in this dataset, my methodology is as follows:

First, I will load the data into a dataframe, removing null values, duplicates, and the target variable, which is in this case 'user_id'. 
Second, I will standardize the data using StandardScaler to prepare it for the PCA.
Third, 

In [1]:
# Initial imports
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly
from sklearn.cluster import KMeans, k_means
from pathlib import Path
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# SQL Alchemy
from sqlalchemy import create_engine

In [2]:
#Loading Data from SQL database
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'ecom'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [3]:
#Examine table data and save into a dataframe
df_shopping = pd.read_sql_query('select * from ecommerce_cb', con=engine)
df_shopping.head()

,com_id,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
0,0,2425083,49125,1,2,18,0.0,17,1,0,13,pantry,baking ingredients
1,1,2425083,49125,1,2,18,0.0,91,2,0,16,dairy eggs,soy lactosefree
2,2,2425083,49125,1,2,18,0.0,36,3,0,16,dairy eggs,butter
3,3,2425083,49125,1,2,18,0.0,83,4,0,4,produce,fresh vegetables
4,4,2425083,49125,1,2,18,0.0,83,5,0,4,produce,fresh vegetables


In [4]:
# List dataframe data types
df_shopping.dtypes

com_id                      int64
order_id                    int64
user_id                     int64
order_number                int64
order_dow                   int64
order_hour_of_day           int64
days_since_prior_order    float64
product_id                  int64
add_to_cart_order           int64
reordered                   int64
department_id               int64
department                 object
product_name               object
dtype: object

In [5]:
#Remove target column from the dataset
# df_shopping = df_shopping.drop(columns=["user_id"])
# df_shopping.head()

In [6]:
# Find null values
df_shopping = df_shopping.dropna()

In [7]:
# Find null values
for column in df_shopping.columns:
    print(f"Column {column} has {df_shopping[column].isnull().sum()} null values")

Column com_id has 0 null values
Column order_id has 0 null values
Column user_id has 0 null values
Column order_number has 0 null values
Column order_dow has 0 null values
Column order_hour_of_day has 0 null values
Column days_since_prior_order has 0 null values
Column product_id has 0 null values
Column add_to_cart_order has 0 null values
Column reordered has 0 null values
Column department_id has 0 null values
Column department has 0 null values
Column product_name has 0 null values


In [8]:
df_shopping.describe()

,com_id,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id
count,2.019501e+06,2.019501e+06,2.019501e+06,2.019501e+06,2.019501e+06,2.019501e+06,2.019501e+06,2.019501e+06,2.019501e+06,2.019501e+06,2.019501e+06
mean,1.009750e+06,1.707013e+06,1.030673e+05,1.715138e+01,2.735367e+00,1.343948e+01,1.068499e+01,7.120590e+01,8.363173e+00,5.897427e-01,9.928349e+00
std,5.829799e+05,9.859832e+05,5.949117e+04,1.752576e+01,2.093882e+00,4.241008e+00,9.111204e+00,3.820727e+01,7.150059e+00,4.918804e-01,6.282933e+00
min,0.000000e+00,1.000000e+01,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00
25%,5.048750e+05,8.526490e+05,5.158400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00,3.100000e+01,3.000000e+00,0.000000e+00,4.000000e+00
50%,1.009750e+06,1.705004e+06,1.026900e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00,8.300000e+01,6.000000e+00,1.000000e+00,9.000000e+00
75%,1.514625e+06,2.559031e+06,1.546000e+05,2.400000e+01,5.000000e+00,1.600000e+01,1.500000e+01,1.070000e+02,1.100000e+01,1.000000e+00,1.600000e+01
max,2.019500e+06,3.421080e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01,1.340000e+02,1.370000e+02,1.000000e+00,2.100000e+01


In [9]:
df_shopping.columns

Index(['com_id', 'order_id', 'user_id', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'department_id', 'department',
       'product_name'],
      dtype='object')

In [12]:
# Scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_shopping[['order_id','user_id','order_dow', 'order_number', 'order_hour_of_day',
       'days_since_prior_order', 'product_id', 'add_to_cart_order',
       'reordered', 'department_id']])

In [13]:
scaled_data.shape

(2019501, 9)

In [14]:
# Applying PCA to reduce dimensions

# Initialize PCA model
pca = PCA(n_components=0.95, random_state = 450)

# Get two principal components for the iris data.
shopping_pca = pca.fit_transform(scaled_data)

In [15]:
#Transform it into a data frame
df_shopping_pca = pd.DataFrame(data=shopping_pca)
df_shopping_pca

,0,1,2,3,4,5,6,7,8
0,-0.603738,-0.718339,0.990831,-0.863745,0.619834,1.011080,-1.903809,-1.041891,-0.244313
1,-0.686448,0.571803,-0.051876,-1.470469,0.531660,0.982212,-1.061480,-1.433131,-0.274440
2,-0.699135,-0.053616,0.739885,-0.987962,0.603883,1.055095,-1.826754,-0.940719,-0.320481
3,-0.630145,-0.590475,0.586785,-0.959214,0.571226,1.078384,0.205543,-1.794151,-0.263772
4,-0.660490,-0.526097,0.641409,-0.904843,0.577016,1.110632,0.261529,-1.736084,-0.295003
...,...,...,...,...,...,...,...,...,...
2019496,-0.408591,-0.275535,0.880105,-0.982828,1.564286,0.907361,0.667588,-1.016506,0.418460
2019497,1.884633,-0.162358,-0.705629,-1.068194,-1.485129,1.069792,0.661944,-0.488508,0.459625
2019498,1.909542,-1.436621,0.566950,-0.282668,-1.374505,1.169805,-0.202799,0.089942,0.424866
2019499,1.879197,-1.372244,0.621574,-0.228297,-1.368715,1.202052,-0.146812,0.148009,0.393635


In [16]:
# Fetch the explained variance
pca.explained_variance_ratio_

array([0.16047557, 0.12045787, 0.1143387 , 0.11358731, 0.11107167,
       0.10886885, 0.10387861, 0.10168395, 0.06563746])